In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [112]:
import numba
import numpy as np
import functools as ft

@numba.jit(nopython=True)
def less_than(xa, ya, xb, yb):
    '''
    Assuming a and b are on adjacent vertices of
    a z-order curve, return -1 if a < b, 0 if
    a == b and 1 if a > b
    
    Parameters
    ----------
    a: (float, float)
        Point
    b: (float, float)
        Point
    '''
    #print("less_than")
        
    ret = 1
    
    if xa == xb and ya == yb:
        return 0
    
    if ya < yb:
        ret = -1
    else:
        if xa < xb:
            ret = -1
    if ya > yb:
        ret = 1
    
    #print(ret)
    return ret
    
    #print(a[1] < b[1], a[0] < b[0], a[1] > b[1], ret)
    #return True if a[1] <= b[1] else a[0] <= b[0]
    #print(a[1] <= b[1], a[0] <= b[0]
    
less_than(0,0, 0,1) # -1 or True
less_than(0,0, 1,0) # -1 or True
less_than(0,0, 1,1) # -1 or True

less_than(1,0, 0,1) # -1 or True
less_than(1,0, 1,1) # -1 or True

less_than(0,1, 1,1) # -1 or True
print("-------------")

less_than(0,1, 0,0) #
less_than(1,0, 0,0) #
less_than(1,1, 0,0) #

less_than(0,1, 1,0) # False
less_than(1,1, 1,0) #

less_than(1,1, 0,1) #

-------------


1

In [113]:
@numba.jit(nopython=True)
def tile_children(a,b,c,d):
    '''
    Split a tile into its four children
    
    Parameters
    ----------
    tile: (float, float, float, float)
    '''
    tile_width = c - a
    tile_height = d - b
    
    return np.array([a, b, a + tile_width / 2, b + tile_height / 2,
            a, b + tile_height / 2, a + tile_width / 2, d,
           a + tile_width / 2, b, c, b + tile_height / 2,
           a + tile_width / 2, b + tile_height / 2, c, d])
        
tile_children(0,0,1,1)


array([ 0. ,  0. ,  0.5,  0.5,  0. ,  0.5,  0.5,  1. ,  0.5,  0. ,  1. ,
        0.5,  0.5,  0.5,  1. ,  1. ])

In [114]:
big_tile=[0,0,8,8]

In [115]:
import math

@numba.jit(nopython=True)
def zindex_compare(a,b,tx0, ty0, tx1, ty1):
    '''
    Compare two points along z curve wihin the bounds of 
    the given tile.
    
    Parameters
    ----------
    a: (float, float)
        Point 1
    b: (float, float)
        Point 2
    tile: (float, float, float, float)
        The bounds of the region in which we want to compare them
    '''
    #print(a, tile, in_tile(a,tile))
    #print("a,b,tile", a,b, tile)
    # assert(in_tile(a,tile))
    # assert(in_tile(b, tile))
    
    #print("comparing", a, b)
    
    if a[0] == b[0] and a[1] == b[1]:
        return 0
    
    tile_width = tx1 - tx0
    tile_height = ty1 - ty0
    
    #children = tile_children(*tile)
    
    quadrant_a = (math.floor( 2 * (a[0] - tx0) / tile_width), 
                        math.floor( 2 * (a[1] - ty0) / tile_height))
    quadrant_b = (math.floor( 2 * (b[0] - tx0) / tile_width), 
                        math.floor( 2 * (b[1] - ty0) / tile_height))
    
    if quadrant_a[0] == quadrant_b[0] and quadrant_a[1] == quadrant_b[1]:
        #print("child_a", child_a, "child_b", child_b)
        x0 = tx0 + quadrant_a[0] * tile_width / 2
        y0 = ty0 + quadrant_a[1] * tile_height / 2
        
        x1 = x0 + tile_width / 2
        y1 = y0 + tile_width / 2
        
        return zindex_compare(a, b, x0,y0,x1,y1)
    else:
        xa = tx0 + quadrant_a[0] * tile_width / 2
        ya = ty0 + quadrant_a[1] * tile_height / 2
        
        xb = tx0 + quadrant_b[0] * tile_width / 2
        yb = ty0 + quadrant_b[1] * tile_height / 2
        
        return less_than(xa,ya,xb,yb)
        #return less_than(child_a[:2], child_b[:2])
        

test_bounds = [0,0,2,2]
new_points = [ [ 0.64297498,  0.53004252], [ 0.64344366,  0.52946568]]
new_points_list = sorted(new_points, key=ft.cmp_to_key(lambda a,b: zindex_compare(a,b, *test_bounds)))

assert(zindex_compare(np.array([2,2]), np.array([2,2]), 0,0,8,8) == 0)
assert(zindex_compare(np.array([0,4]), np.array([3,4]), 0,0,8,8) == -1)
assert(zindex_compare([6,4], [3,1], 0,0,8,8) == 1)
assert(zindex_compare([3,1], [2,2], 0,0,8,8) == -1)
assert(zindex_compare([2,2], [3,1], 0,0,8,8) == 1)
assert(zindex_compare([7,1], [2,2], 0,0,8,8) == 1)
assert(zindex_compare([2,2], [7,1], 0,0,8,8) == -1)

In [116]:
%%time

points_list = sorted([
    [6,4],[7,1],[2,2],[3,1],[3,4],[0,4],[1,6],[4,2]
], key=ft.cmp_to_key(lambda a,b: zindex_compare(a,b, 0,0,8,8)))

print(points_list)

[[3, 1], [2, 2], [7, 1], [4, 2], [0, 4], [3, 4], [1, 6], [6, 4]]
CPU times: user 305 µs, sys: 544 µs, total: 849 µs
Wall time: 545 µs


In [117]:
def lower_bound(sequence, value, compare=None):
    """Find the index of the first element in sequence >= value"""
    elements = len(sequence)
    offset = 0
    middle = 0
    found = len(sequence)
 
    while elements > 0:
        middle = elements // 2
        #print("middle:", middle)
        if compare(value, sequence[offset + middle]) > 0:
            offset = offset + middle + 1
            elements = elements - (middle + 1)
        else:
            found = offset + middle
            elements = middle
    return found

def upper_bound(sequence, value, compare):
    """Find the index of the first element in sequence > value"""
    elements = len(sequence)
    offset = 0
    middle = 0
    found = 0
 
    while elements > 0:
        middle = elements // 2
        if compare(value, sequence[offset + middle]) < 0:
            elements = middle
        else:
            offset = offset + middle + 1
            found = offset
            elements = elements - (middle + 1)
    return found

print(lower_bound(points_list, [1,6], lambda a,b: zindex_compare(a,b, 0,0,8,8)))
print(upper_bound(points_list, [1,6], lambda a,b: zindex_compare(a,b, 0,0,8,8)))

6
7


In [118]:
def all_point_boundaries(points_list, tile, bounding_tile):
    cmp = lambda a,b: zindex_compare(a,b, *bounding_tile)
    
    left_index = lower_bound(points_list, [tile[0], tile[1]], cmp)
    right_index = lower_bound(points_list, [tile[2] - 0.000001, tile[3] - 0.000001], cmp)
    #print("right_index", right_index)
    
    return left_index, right_index

def all_points(points_list, tile, bounding_tile):
    '''
    Return all points that are in the tile given tile
    '''
    left_index, right_index = all_point_boundaries(points_list, tile, bounding_tile)
    
    return points_list[left_index:right_index]

all_points(points_list, [0.0, 0.0, 4.0, 4.0], big_tile)

[[3, 1], [2, 2]]

In [119]:
#all_points(points_list, [3.0, 1.9999999999999998, 3.0000000000000004, 2.0]

In [120]:
def all_in(tile, rect):
    '''
    Is this tile completely enclosed in the rectangle:
    
    Parameters
    ----------
    tile: [float, float, float, float]
        The xmin,ymin,xmax,ymax of the tile
    rect: [float, float, float, float]
        The xmin, ymin, xmas, ymax of the rectangle
    '''
    ret = (tile[0] >= rect[0] and
            tile[1] >= rect[1] and 
            tile[2] <= rect[2] and
            tile[3] <= rect[3])
    #print("all_in:", ret, tile, rect)
    return ret

def some_in(tile, rect):
    '''
    Is part of this tile within the rectangle?
    
    Parameters
    ----------
    tile: [float, float, float, float]
        The xmin,ymin,xmax,ymax of the tile
    rect: [float, float, float, float]
        The xmin, ymin, xmas, ymax of the tile
    '''
    ret = (tile[0] < rect[2] and
            tile[1] < rect[3] and 
            tile[2] > rect[0] and
            tile[3] >= rect[1])
    
    #print("some_in", ret)
    return ret

def get_points(points_list, rect, tile, bounding_tile):
    '''
    Get all the points in the tile that intersect the rectangle
    
    Parameters
    ----------
    rect: [float, float, float, float]
        minx, miny, maxx, maxy
    tile: [float, float, float, float]
        minx, miny, maxx, maxy
    '''
    points = []
    #print("tile:", tile)
    tiles_to_check = [tile]
    
    while len(tiles_to_check) > 0:
        tile = tiles_to_check.pop()
        
        left_index, right_index = all_point_boundaries(points_list, tile, bounding_tile)
    
    
        #if len(points_in_tile) == 0:
        #    return []
    
        #print("tile:", tile)
        # points_in_tile
        if left_index == right_index:
            continue
    
        if not some_in(tile, rect):
            # no intersection
            continue
            
        if all_in(tile, rect):
            #print("all points", tile)
            points += points_list[left_index:right_index]
            continue
    
        for child in np.array(tile_children(*tile)).reshape((4,-1)):
            tiles_to_check += [child]
            #points += get_points(points_list, rect, child, bounding_tile) 
        
    return points

In [121]:
get_points(points_list, [0,0,4,4], [0,0,8,8], [0,0,16,16])

[[3, 1], [2, 2]]

In [122]:
get_points(points_list, [0,0,5,5], [0,0,8,8], [0,0,16,16])

[[4, 2], [3, 4], [0, 4], [3, 1], [2, 2]]

In [123]:
get_points(points_list, [0,0,8,1.5], [0,0,8,8], [0,0,16,16])

[[7, 1], [3, 1]]

In [124]:
new_points = [ [ 0.64297498,  0.53004252], [ 0.64344366,  0.52946568]]

new_points_list = sorted(new_points, key=ft.cmp_to_key(lambda a,b: zindex_compare(a,b, *test_bounds)))

In [125]:
len(new_points_list)

2

In [133]:
%time new_points = np.random.random((256000,2))
%time sorted(new_points, key=ft.cmp_to_key(lambda a,b: a[0] - b[0]))
test_bounds = [0,0,2,2]

%time new_points_list = sorted(new_points, key=ft.cmp_to_key(ft.partial(zindex_compare,tx0= test_bounds[0],ty0=test_bounds[1], tx1=test_bounds[2], ty1=test_bounds[3])))

CPU times: user 10.4 ms, sys: 3.45 ms, total: 13.9 ms
Wall time: 12.9 ms
CPU times: user 2.8 s, sys: 30.8 ms, total: 2.83 s
Wall time: 2.86 s
CPU times: user 5.6 s, sys: 50.5 ms, total: 5.65 s
Wall time: 5.67 s


In [143]:
import random

def run_query_z_index():
    #print(query_bounds)
    min_pos = [random.random(), random.random()]
    a = 1 - min_pos[0]
    b = 1 - min_pos[1]
    max_pos = [min_pos[0] + a * random.random(), min_pos[1] + b * random.random()]
    #print(min_pos, max_pos)
    query_bounds = min_pos + max_pos
    
    return get_points(new_points_list, query_bounds, test_bounds, test_bounds)

#def naive_filter():
    
import timeit

#%timeit run_query_z_index()
timeit.timeit(run_query_z_index, number=1)
#print(len(x), query_bounds)

KeyboardInterrupt: 

In [111]:
times = np.array([
    [1000, 17, 14],
    [2000, 29, 24],
    [4000, 81, 35],
    [8000, 144, 55],
    [16000, 251, 102],
    [32000, 525, 140],
    [64000, 1140, 242],
    [128000, 2600, 315],
    [256000, 5690,  ],
    [512000, ]
])

In [ ]:
times1 = np.array([
    [1000, 159, 118],
    [2000, 374, 217],
    [4000, 810, 281],
    [8000, 1740, 462],
    [16000, 3880, 1000],
    [32000, 8810, 1350],
    [64000, 19800, 1470],
    [132000, 45000, 2830]
])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(times[:,1], times[:,2])

In [ ]:
%timeit x(78,81)

In [ ]:
%timeit add(78,81)

In [ ]:
%timeit z(78, 81)